In [ ]:
#Importing all the libraries
import os
import numpy as np
import matplotlib.pyplot as plt
#Keras
import tensorflow.keras
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Add, Input, BatchNormalization, Activation
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.models import load_model
#Scikit-Learn
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#Defining training and validation directories
train_data_dir=r'dataset2'
val_data_dir=r'dataset2\cloudy'

# Defining Data Generators

In [ ]:
target_size = (250, 250)
batch_size = 32

#Data Generator for training samples
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=15,
                                   fill_mode='reflect',
                                   data_format='channels_last',
                                   brightness_range=[0.5, 1.5])

#Data Generator for validation samples with no augmentation
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

validation_generator = test_datagen.flow_from_directory(val_data_dir,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

# BUILDING CNN ARCHITECTURE

In [ ]:
class Architecture:
    
    @staticmethod
    def build(width, height, depth, classes, finalAct="softmax"):
        
        """
        This function builds a sequential CNN model and returns the model

        Parameters:
        width: width of the input image
        height: height of the input image
        depth: Number of channels
        classes: Number of classes to classify
        finalAct: THe activation funtion for the output node
        """
        
        inputShape = (height, width, depth)
        chanDim = -1

        #Checking for data_format
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        #Initializing the he_uniform kernel for weights
        initializer=tensorflow.keras.initializers.he_uniform(seed=200)

        #Sequential Model Building
        model = Sequential()

        #BLOCK 1
        model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'same',
                         input_shape = inputShape, kernel_initializer=initializer))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Activation("relu"))
        model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))

        #BLOCK 2
        model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Activation("relu"))
        model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

        #BLOCK 3
        model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Activation("relu"))
        model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'same',kernel_initializer=initializer))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

        #BLOCK 4 
        model.add(Conv2D(filters = 256, kernel_size = (5,5),padding = 'same',kernel_initializer=initializer))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Activation("relu"))
        model.add(Conv2D(filters = 256, kernel_size = (5,5),padding = 'same',kernel_initializer=initializer))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

        #Classifier
        model.add(GlobalAveragePooling2D())
        model.add(Dense(512, activation = "relu",kernel_initializer=initializer))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation = "relu",kernel_initializer=initializer))
        model.add(Dropout(0.5))
        model.add(Dense(classes, activation = finalAct))

        return model

In [ ]:
#initialize the number of epochs to train for, initial learning rate, batch size, and image dimensions
epochs = 100
batch_size = 16
img_dims = (250, 250, 3)

In [ ]:
#Calling build function to build a sequential model
model = Architecture.build(
    width=img_dims[0], 
    height=img_dims[1],
    depth=img_dims[2], 
    classes=4)

In [ ]:
model.summary()

In [ ]:
#Saving the model structure to visualize
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
#Using Adam optimizer with an initial learning rate of 0.0001
opt=Adam(learning_rate=0.001, beta_1=0.91, beta_2=0.999, epsilon=1e-08, decay=0.0)
#Compile the model
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
#Defining Callbacks
def myCallbacks():
    """This function returns a list of callbacks"""

    #Model Checkpoint
    # file_path = r"/content/drive/My Drive/weather_classification/MODEL.h5"
    # checkpoint = callbacks.ModelCheckpoint(file_path,monitor='val_acc',verbose=1,save_best_only=True,mode='auto')
    #ReduceLROnPlateau
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_acc',factor=0.1,patience=5,min_lr=1e-30,cooldown=3,verbose=1)
    #EarlyStopping
    #es = callbacks.EarlyStopping(monitor='val_acc', mode='min', verbose=1, patience=15)
    return [reduce_lr]

In [ ]:
#Training the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.n// batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.n// batch_size,
    epochs = epochs,
    verbose=1,
    callbacks=myCallbacks()
    )

# Model Evaluation

In [ ]:
history.history

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model.evaluate_generator(train_generator, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)

So the maximum accuracy i got 95% with the minimum loss of 0.11

In [ ]:
#Saving the model
model.save("/content/drive/My Drive/weather_classification/final_model.h5")

# Classification Report

In [ ]:
#Defining a test data generator for making predictions on the complete test folder
test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = '/content/drive/My Drive/weather_classification/weatherDataset.zip (Unzipped Files)/weatherDataset/val'
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(250, 250),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)

In [ ]:
#Making predictions
predict = model.predict_generator(test_generator,steps = nb_samples)

In [ ]:
#Getting predicted classes with higher probability using argmax 
predicted_classes = predict.argmax(axis=-1)

In [ ]:
#Classification Report
print(classification_report(test_generator.classes, predicted_classes))

In [ ]:
#Confusion matrix
confusion_matrix(test_generator.classes, predicted_classes)

# Predictions
## I have downloaded random images from web for making predictions

In [ ]:
#Loading the model to make predictions
model = load_model('/content/drive/My Drive/weather_classification/final_model.h5')

In [ ]:
def predict(path):
    """This functions returns predictions for a given input image

    Parameters: 
    path: path of the image to predict

    """
    #Creating a dictionary of classes as the predict function gives probablities
    actualClasses = { 0:'Cloudy',1:'Rain',2:'Shine',3:'Sunrise' }

    img = load_img(path, target_size=(250, 250))
    img_array = img_to_array(img)

    #Plotting the image
    plt.imshow(img_array/255)
    plt.show()
    img_array = np.expand_dims(img_array, axis=0)

    #Prediction
    pred = model.predict(img_array)
    classes = np.argmax(pred)
    return 'Predicted Class for the input image : {}'.format(actualClasses[classes])

In [ ]:
!wget https://pramanaexperience.com/wp-content/uploads/2019/09/mount-batur-sunrise-trekking.jpg

In [ ]:
predict(path="/content/mount-batur-sunrise-trekking.jpg")

In [ ]:
!wget https://c.ndtvimg.com/2019-09/psjsc54_heavy-rain-in-mumbai_625x300_04_September_19.jpg

In [ ]:
predict(path="/content/psjsc54_heavy-rain-in-mumbai_625x300_04_September_19.jpg")

In [ ]:
!wget https://miro.medium.com/max/10830/0*c7Q-GSzNAEg3V2Fu

In [ ]:
predict(path="/content/0*c7Q-GSzNAEg3V2Fu")

In [ ]:
!wget http://e993.com/forex/imgs/gabans.files.wordpress.com/2012/04/dark-clouds.jpg

In [ ]:
predict(path="/content/dark-clouds.jpg")

In [ ]:
!wget https://images.pond5.com/dark-cloudy-day-brooklyn-neighborhood-footage-082056671_iconl.jpeg

In [ ]:
predict(path="/content/dark-cloudy-day-brooklyn-neighborhood-footage-082056671_iconl.jpeg")

# Conclusion
Trained A CNN classifer from scratch to classify Weather Images with validation accuracy=95.796%